In [1]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.9/316.9 MB 2.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.0-py2.py3-none-any.whl size=317425345 sha256=25cb3194181bf6c3d34002fd8cdc75f6a071b8fa31ac3d705e1da8d46bda7a96
  Stored in directory: /root/.cache/pip/wheels/41/4e/10/c2cf2467f71c678cfc8a6b9ac9241e5e44a01940da8fbb17fc
Successfully built pyspark


In [2]:
from pyspark.sql import SparkSession, Window
from pyspark.sql.functions import *

In [3]:
spark = SparkSession.builder.appName("case_study_eda").master("local[*]").getOrCreate()

In [5]:
data_source_path = 'data/Data/'
df_charge = spark.read.csv(data_source_path+"Charges_use.csv", header=True, inferSchema=True)
df_damage = spark.read.csv(data_source_path+"Damages_use.csv", header=True, inferSchema=True)
df_endorse = spark.read.csv(data_source_path+"Endorse_use.csv", header=True, inferSchema=True)
df_person = spark.read.csv(data_source_path+"Primary_Person_use.csv", header=True, inferSchema=True)
df_restrict = spark.read.csv(data_source_path+"Restrict_use.csv", header=True, inferSchema=True)
df_units = spark.read.csv(data_source_path+"Units_use.csv", header=True, inferSchema=True)

# Charge Table

In [ ]:
df_charge.show(10)

+--------+--------+--------+--------------------+------------+
|CRASH_ID|UNIT_NBR|PRSN_NBR|              CHARGE|CITATION_NBR|
+--------+--------+--------+--------------------+------------+
|14768622|       1|       1|DRIVING WHILE INT...|        NULL|
|14838637|       1|       1|                 DWI|  1600000015|
|14838641|       1|       1|RAN RED LIGHT SOL...|      L20440|
|14838641|       2|       1|NO DRIVER'S LICEN...|      L23141|
|14838668|       1|       1|DRIVING WHILE INT...|TX4IC50SRJD3|
|14838669|       2|       1|     DWI W/BAC >.015| 2015-000006|
|14838670|       1|       1|DRIVING WHILE INT...| 2016-000003|
|14838685|       1|       1|FAILED TO DRIVE S...|   138434825|
|14838693|       1|       1|DRIVING WHILE INT...|TX4IC60UKQND|
|14838768|       2|       1|                 DWI|        NULL|
+--------+--------+--------+--------------------+------------+
only showing top 10 rows



In [ ]:
df_charge.printSchema()

root
 |-- CRASH_ID: integer (nullable = true)
 |-- UNIT_NBR: integer (nullable = true)
 |-- PRSN_NBR: integer (nullable = true)
 |-- CHARGE: string (nullable = true)
 |-- CITATION_NBR: string (nullable = true)



In [7]:
df_charge.count()

116110

In [8]:
df_charge.select('crash_id','unit_nbr','prsn_nbr').distinct().count()

93415

In [15]:
df_damage.where(col("crash_id").isNull() | col("crash_id").isin('NA')).count()

0

# Damage Table

In [9]:
df_damage.show(10)

+--------+--------------------+
|CRASH_ID|    DAMAGED_PROPERTY|
+--------+--------------------+
|14768622|             MAILBOX|
|14768622|         YARD, GRASS|
|14838668|           GUARDRAIL|
|14838685|           ROAD SIGN|
|14838693|        2009 MAZDA 3|
|14838834|    CHAIN LINK FENCE|
|14838841|WOODED POLE ON SO...|
|14838842|CITY SIGN FOR TUR...|
|14838877|    FENCE-CHAIN LINK|
|14838977|LANDSCAPING AND M...|
+--------+--------------------+
only showing top 10 rows



In [18]:
df_damage.printSchema()

root
 |-- CRASH_ID: integer (nullable = true)
 |-- DAMAGED_PROPERTY: string (nullable = true)



In [11]:
df_damage.count()

24950

In [ ]:
df_damage.count()

24950

In [12]:
df_damage.select('crash_id').distinct().count()

20887

In [16]:
df_damage.where(col("crash_id").isNull() | col("crash_id").isin('NA')).count()

0

# Endorse Table

In [17]:
df_endorse.show(10)

+--------+--------+------------------+
|CRASH_ID|UNIT_NBR|DRVR_LIC_ENDORS_ID|
+--------+--------+------------------+
|14768622|       1|              NONE|
|14838637|       1|              NONE|
|14838637|       2|              NONE|
|14838641|       1|              NONE|
|14838641|       2|        UNLICENSED|
|14838668|       1|              NONE|
|14838669|       1|              NONE|
|14838669|       2|              NONE|
|14838670|       1|              NONE|
|14838685|       1|        UNLICENSED|
+--------+--------+------------------+
only showing top 10 rows



In [19]:
df_endorse.printSchema()

root
 |-- CRASH_ID: integer (nullable = true)
 |-- UNIT_NBR: integer (nullable = true)
 |-- DRVR_LIC_ENDORS_ID: string (nullable = true)



In [24]:
df_endorse.select('drvr_lic_endors_id').distinct().show(truncate=False)

+-------------------------------------+
|drvr_lic_endors_id                   |
+-------------------------------------+
|SCHOOL BUS                           |
|OTHER/OUT OF STATE                   |
|UNKNOWN                              |
|HAZARDOUS MATERIALS                  |
|UNLICENSED                           |
|TANK VEHICLE                         |
|NONE                                 |
|TANK VEHICLE WITH HAZARDOUS MATERIALS|
|PASSENGER                            |
|DOUBLE/TRIPLE TRAILER                |
+-------------------------------------+



In [25]:
df_endorse.count()

159818

In [21]:
df_endorse.select('crash_id','unit_nbr').distinct().count()

156495

In [23]:
df_endorse.where(col("crash_id").isNull() | col("crash_id").isin('NA')).count()

0

In [31]:
df_endorse.select('unit_nbr').distinct().orderBy(col('unit_nbr').desc()).show()

+--------+
|unit_nbr|
+--------+
|      14|
|      13|
|      12|
|      11|
|      10|
|       9|
|       8|
|       7|
|       6|
|       5|
|       4|
|       3|
|       2|
|       1|
+--------+



# Person Table

In [34]:
df_person.show(10,truncate=False)

+--------+--------+--------+------------+------------------+-------------------------+--------+-----------------+------------+------------+-------------------+-----------------+--------------+-----------+---------------------+----------------+------------------+---------------------+----------------+-----------------+---------------+---------------+------------------+--------------+-------------+--------------+-------------+---------+----------------------+-----------------+------------------+--------+
|CRASH_ID|UNIT_NBR|PRSN_NBR|PRSN_TYPE_ID|PRSN_OCCPNT_POS_ID|PRSN_INJRY_SEV_ID        |PRSN_AGE|PRSN_ETHNICITY_ID|PRSN_GNDR_ID|PRSN_EJCT_ID|PRSN_REST_ID       |PRSN_AIRBAG_ID   |PRSN_HELMET_ID|PRSN_SOL_FL|PRSN_ALC_SPEC_TYPE_ID|PRSN_ALC_RSLT_ID|PRSN_BAC_TEST_RSLT|PRSN_DRG_SPEC_TYPE_ID|PRSN_DRG_RSLT_ID|DRVR_DRG_CAT_1_ID|PRSN_DEATH_TIME|INCAP_INJRY_CNT|NONINCAP_INJRY_CNT|POSS_INJRY_CNT|NON_INJRY_CNT|UNKN_INJRY_CNT|TOT_INJRY_CNT|DEATH_CNT|DRVR_LIC_TYPE_ID      |DRVR_LIC_STATE_ID|DRVR_LIC_CLS_

In [33]:
df_person.printSchema()

root
 |-- CRASH_ID: integer (nullable = true)
 |-- UNIT_NBR: integer (nullable = true)
 |-- PRSN_NBR: integer (nullable = true)
 |-- PRSN_TYPE_ID: string (nullable = true)
 |-- PRSN_OCCPNT_POS_ID: string (nullable = true)
 |-- PRSN_INJRY_SEV_ID: string (nullable = true)
 |-- PRSN_AGE: string (nullable = true)
 |-- PRSN_ETHNICITY_ID: string (nullable = true)
 |-- PRSN_GNDR_ID: string (nullable = true)
 |-- PRSN_EJCT_ID: string (nullable = true)
 |-- PRSN_REST_ID: string (nullable = true)
 |-- PRSN_AIRBAG_ID: string (nullable = true)
 |-- PRSN_HELMET_ID: string (nullable = true)
 |-- PRSN_SOL_FL: string (nullable = true)
 |-- PRSN_ALC_SPEC_TYPE_ID: string (nullable = true)
 |-- PRSN_ALC_RSLT_ID: string (nullable = true)
 |-- PRSN_BAC_TEST_RSLT: string (nullable = true)
 |-- PRSN_DRG_SPEC_TYPE_ID: string (nullable = true)
 |-- PRSN_DRG_RSLT_ID: string (nullable = true)
 |-- DRVR_DRG_CAT_1_ID: string (nullable = true)
 |-- PRSN_DEATH_TIME: timestamp (nullable = true)
 |-- INCAP_INJRY_CNT: 

In [40]:
df_person.select('prsn_type_id').distinct().show(truncate=False)

+---------------------------------+
|prsn_type_id                     |
+---------------------------------+
|PEDESTRIAN                       |
|DRIVER OF MOTORCYCLE TYPE VEHICLE|
|PASSENGER/OCCUPANT               |
|DRIVER                           |
|UNKNOWN                          |
|OTHER (EXPLAIN IN NARRATIVE)     |
|PEDALCYCLIST                     |
+---------------------------------+



In [35]:
df_person.select('prsn_occpnt_pos_id').distinct().show(truncate=False)

+-------------------------------------------------+
|prsn_occpnt_pos_id                               |
+-------------------------------------------------+
|FRONT CENTER                                     |
|SECOND SEAT LEFT                                 |
|PEDESTRIAN, PEDALCYCLIST, OR MOTORIZED CONVEYANCE|
|FRONT LEFT                                       |
|CARGO AREA                                       |
|UNKNOWN                                          |
|SECOND SEAT CENTER                               |
|PASSENGER IN BUS                                 |
|THIRD SEAT LEFT                                  |
|OTHER  (EXPLAIN IN NARRATIVE)                    |
|FRONT RIGHT                                      |
|OTHER IN VEHICLE                                 |
|SECOND SEAT RIGHT                                |
|OUTSIDE VEHICLE                                  |
+-------------------------------------------------+



In [36]:
df_person.select('prsn_injry_sev_id').distinct().show(truncate=False)

+-------------------------+
|prsn_injry_sev_id        |
+-------------------------+
|NA                       |
|KILLED                   |
|UNKNOWN                  |
|NON-INCAPACITATING INJURY|
|NOT INJURED              |
|POSSIBLE INJURY          |
|INCAPACITATING INJURY    |
+-------------------------+



In [37]:
df_person.select('prsn_ethnicity_id').distinct().show(truncate=False)

+---------------------------+
|prsn_ethnicity_id          |
+---------------------------+
|WHITE                      |
|BLACK                      |
|HISPANIC                   |
|NA                         |
|AMER. INDIAN/ALASKAN NATIVE|
|UNKNOWN                    |
|OTHER                      |
|ASIAN                      |
+---------------------------+



In [39]:
df_person.select('prsn_gndr_id').distinct().show(truncate=False)

+------------+
|prsn_gndr_id|
+------------+
|NA          |
|UNKNOWN     |
|MALE        |
|FEMALE      |
+------------+



In [38]:
df_person.select('prsn_ejct_id').distinct().show(truncate=False)

+--------------+
|prsn_ejct_id  |
+--------------+
|YES, PARTIAL  |
|NA            |
|YES           |
|UNKNOWN       |
|NOT APPLICABLE|
|NO            |
+--------------+



In [41]:
df_person.select('prsn_rest_id').distinct().show(truncate=False)

+----------------------------+
|prsn_rest_id                |
+----------------------------+
|NA                          |
|CHILD SEAT, UNKNOWN         |
|LAP BELT ONLY               |
|UNKNOWN                     |
|SHOULDER BELT ONLY          |
|NOT APPLICABLE              |
|OTHER (EXPLAIN IN NARRATIVE)|
|CHILD BOOSTER SEAT          |
|NONE                        |
|SHOULDER & LAP BELT         |
|CHILD SEAT, FACING REAR     |
|CHILD SEAT, FACING FORWARD  |
+----------------------------+



In [42]:
df_person.select('prsn_airbag_id').distinct().show(truncate=False)

+-----------------+
|prsn_airbag_id   |
+-----------------+
|NA               |
|DEPLOYED, REAR   |
|NOT DEPLOYED     |
|DEPLOYED, SIDE   |
|UNKNOWN          |
|NOT APPLICABLE   |
|DEPLOYED MULTIPLE|
|DEPLOYED, FRONT  |
+-----------------+



In [43]:
df_person.select('prsn_helmet_id').distinct().show(truncate=False)

+-----------------+
|prsn_helmet_id   |
+-----------------+
|WORN, UNK DAMAGE |
|WORN, DAMAGED    |
|NOT WORN         |
|NOT APPLICABLE   |
|WORN, NOT DAMAGED|
|UNKNOWN IF WORN  |
+-----------------+



In [44]:
df_person.select('prsn_alc_rslt_id').distinct().show(truncate=False)

+----------------+
|prsn_alc_rslt_id|
+----------------+
|NA              |
|Positive        |
|Negative        |
+----------------+



In [45]:
df_person.select('drvr_lic_type_id').distinct().show(truncate=False)

+----------------------+
|drvr_lic_type_id      |
+----------------------+
|NA                    |
|COMMERCIAL DRIVER LIC.|
|ID CARD               |
|UNKNOWN               |
|OCCUPATIONAL          |
|UNLICENSED            |
|OTHER                 |
|DRIVER LICENSE        |
+----------------------+



In [47]:
df_person.select('drvr_lic_state_id').distinct().show(truncate=False)

+------------------------+
|drvr_lic_state_id       |
+------------------------+
|Utah                    |
|Hawaii                  |
|Minnesota               |
|Ohio                    |
|Northern Mariana Islands|
|Arkansas                |
|Oregon                  |
|NA                      |
|Texas                   |
|North Dakota            |
|Pennsylvania            |
|Connecticut             |
|Nebraska                |
|Vermont                 |
|Nevada                  |
|Puerto Rico             |
|Washington              |
|Illinois                |
|Oklahoma                |
|Unknown                 |
+------------------------+
only showing top 20 rows



In [48]:
df_person.select('drvr_lic_cls_id').distinct().show(truncate=False)

+------------------+
|drvr_lic_cls_id   |
+------------------+
|CLASS C           |
|CLASS C AND M     |
|NA                |
|OTHER/OUT OF STATE|
|CLASS B           |
|CLASS A AND M     |
|CLASS M           |
|CLASS A           |
|UNKNOWN           |
|CLASS B AND M     |
|UNLICENSED        |
+------------------+



In [ ]:
df_person.count()

156954

In [50]:
df_person.distinct().count()

156954

In [49]:
df_person.select('crash_id','unit_nbr','prsn_nbr').distinct().count()

156954

# Restrict Table

In [51]:
df_restrict.show(10)

+--------+--------+--------------------+
|CRASH_ID|UNIT_NBR| DRVR_LIC_RESTRIC_ID|
+--------+--------+--------------------+
|14768622|       1|                NONE|
|14838637|       1|  OTHER/OUT OF STATE|
|14838637|       2|                NONE|
|14838641|       1|WITH CORRECTIVE L...|
|14838641|       2|          UNLICENSED|
|14838668|       1|                NONE|
|14838669|       1|                NONE|
|14838669|       2|WITH CORRECTIVE L...|
|14838670|       1|                NONE|
|14838685|       1|          UNLICENSED|
+--------+--------+--------------------+
only showing top 10 rows



In [52]:
df_restrict.printSchema()

root
 |-- CRASH_ID: integer (nullable = true)
 |-- UNIT_NBR: integer (nullable = true)
 |-- DRVR_LIC_RESTRIC_ID: string (nullable = true)



In [53]:
df_restrict.select('drvr_lic_restric_id').distinct().show(truncate=False)

+---------------------------------------------------+
|drvr_lic_restric_id                                |
+---------------------------------------------------+
|MOPED (EFF. END 12/31/2016)                        |
|APPLICABLE PROSTHETIC DEVICES                      |
|FRSI CDL MM/DD/YY - MM/DD/YY OR EXEMPT A VEH       |
|TRC 545.424 APPLIES UNTIL MM/DD/YY                 |
|TO/FROM WORK OR LOFS 21 OR OVER                    |
|INTRASTATE ONLY (EFF. 1/1/2017)                    |
|WITH CORRECTIVE LENSES                             |
|STATED ON LICENSE                                  |
|OPERATION CLASS A EXEMPT VEH AUTHORIZED            |
|VEHICLE NOT TO EXCEED CLASS C (EFF. END 12/31/2016)|
|NO CLASS A AND B PASSENGER VEHICLE (EFF. 1/1/2017) |
|IF CMV, CUSTOM-HARVESTING INTERSTATE               |
|LOFS 21 OR OVER SCHOOL BUS ONLY                    |
|PASSENGER CMVS RESTRICT TO CLASS C ONLY            |
|IF CMV, PRIVATELY TRANS PASSENGERS INTERSTATE      |
|OPERATION CLASS B EXEMPT VE

In [54]:
df_restrict.count()

159852

In [55]:
df_restrict.distinct().count()

159852

In [56]:
df_restrict.select('crash_id','unit_nbr').distinct().count()

156500

# Unit Table

In [57]:
df_units.show(10)

+--------+--------+-------------+-------------+----------+----------------+-----------------+------------+------------+------------+---------------+--------------------+---------------+--------+-----------------+--------------------+--------------------+-----------------+--------------+--------------------+-----------------+--------------+------------------+--------------------+--------------------+--------------------+--------------------+-------------------+---------------+---------------------+---------------+------------------+--------------+-------------+--------------+-------------+---------+
|CRASH_ID|UNIT_NBR| UNIT_DESC_ID|VEH_PARKED_FL|VEH_HNR_FL|VEH_LIC_STATE_ID|              VIN|VEH_MOD_YEAR|VEH_COLOR_ID| VEH_MAKE_ID|     VEH_MOD_ID|    VEH_BODY_STYL_ID|EMER_RESPNDR_FL|OWNR_ZIP|FIN_RESP_PROOF_ID|    FIN_RESP_TYPE_ID|  VEH_DMAG_AREA_1_ID|VEH_DMAG_SCL_1_ID|FORCE_DIR_1_ID|  VEH_DMAG_AREA_2_ID|VEH_DMAG_SCL_2_ID|FORCE_DIR_2_ID|VEH_INVENTORIED_FL|     VEH_TRANSP_NAME|     VEH_TRAN

In [58]:
df_units.printSchema()

root
 |-- CRASH_ID: integer (nullable = true)
 |-- UNIT_NBR: integer (nullable = true)
 |-- UNIT_DESC_ID: string (nullable = true)
 |-- VEH_PARKED_FL: string (nullable = true)
 |-- VEH_HNR_FL: string (nullable = true)
 |-- VEH_LIC_STATE_ID: string (nullable = true)
 |-- VIN: string (nullable = true)
 |-- VEH_MOD_YEAR: string (nullable = true)
 |-- VEH_COLOR_ID: string (nullable = true)
 |-- VEH_MAKE_ID: string (nullable = true)
 |-- VEH_MOD_ID: string (nullable = true)
 |-- VEH_BODY_STYL_ID: string (nullable = true)
 |-- EMER_RESPNDR_FL: string (nullable = true)
 |-- OWNR_ZIP: string (nullable = true)
 |-- FIN_RESP_PROOF_ID: string (nullable = true)
 |-- FIN_RESP_TYPE_ID: string (nullable = true)
 |-- VEH_DMAG_AREA_1_ID: string (nullable = true)
 |-- VEH_DMAG_SCL_1_ID: string (nullable = true)
 |-- FORCE_DIR_1_ID: string (nullable = true)
 |-- VEH_DMAG_AREA_2_ID: string (nullable = true)
 |-- VEH_DMAG_SCL_2_ID: string (nullable = true)
 |-- FORCE_DIR_2_ID: string (nullable = true)
 |--

In [59]:
df_units.count()

173499

In [60]:
df_units.distinct().count()

168124

In [63]:
df_units.dropDuplicates(['crash_id','unit_nbr']).count()

168020

In [62]:
df_units.dropDuplicates(['crash_id','unit_nbr','unit_desc_id']).count()

168020

In [64]:
df_units.select('unit_desc_id').distinct().show(truncate=False)

+-----------------------------+
|unit_desc_id                 |
+-----------------------------+
|PEDESTRIAN                   |
|NON-CONTACT                  |
|TRAIN                        |
|MOTOR VEHICLE                |
|MOTORIZED CONVEYANCE         |
|OTHER  (EXPLAIN IN NARRATIVE)|
|PEDALCYCLIST                 |
|TOWED/TRAILER                |
+-----------------------------+



In [65]:
df_units.select('veh_hnr_fl').distinct().show(truncate=False)

+----------+
|veh_hnr_fl|
+----------+
|Y         |
|N         |
|NULL      |
+----------+



In [66]:
df_units.select('veh_lic_state_id').distinct().show(truncate=False)

+----------------+
|veh_lic_state_id|
+----------------+
|AZ              |
|SC              |
|LA              |
|MN              |
|NJ              |
|MX              |
|DC              |
|OR              |
|NA              |
|98              |
|VA              |
|RI              |
|KY              |
|WY              |
|NH              |
|MI              |
|NV              |
|WI              |
|ID              |
|CA              |
+----------------+
only showing top 20 rows



In [67]:
df_units.select('vin').distinct().show(truncate=False)

+-----------------+
|vin              |
+-----------------+
|1FAFP53U85A190865|
|4T1BG22KX1U857396|
|1N4AA5AP5CC862603|
|JT4VN93D5M5023678|
|3GCEC13JX7G523359|
|3C4FY58B95T505146|
|1ZVFT82H965215324|
|JN1AZ34D45M604758|
|1FAFP5357XA316506|
|5TFRM5F14CX043017|
|1GTEK19ZX5Z309758|
|3GNEC16Z52G176864|
|1GCEC14V52Z217681|
|1GNFC13J57R299137|
|1GNEC13T41J143853|
|1FAFP42X1YF275608|
|1FTFW1EF7DKF49488|
|1FMCU93178KE69253|
|1LNHM82WX3Y653525|
|4T1BK1EBXEU132889|
+-----------------+
only showing top 20 rows



In [71]:
df_units.filter(col('vin').isNull() | col('vin').isin('NA')).count()

7699

In [70]:
df_units.select('veh_mod_year').distinct().show(truncate=False)

+------------+
|veh_mod_year|
+------------+
|1957        |
|1987        |
|1936        |
|1958        |
|1972        |
|1988        |
|1977        |
|1971        |
|1984        |
|1982        |
|2005        |
|1919        |
|NA          |
|2000        |
|1965        |
|1981        |
|1940        |
|1978        |
|1974        |
|2002        |
+------------+
only showing top 20 rows



In [72]:
df_units.filter(col('veh_mod_year').isNull() | col('veh_mod_year').isin('NA')).count()

75947

In [75]:
df_units.select('veh_color_id').distinct().show(truncate=False)

+------------+
|veh_color_id|
+------------+
|RED         |
|TEA         |
|BLK         |
|ONG         |
|YEL         |
|GLD         |
|NA          |
|98          |
|99          |
|CAM         |
|BRZ         |
|TRQ         |
|SIL         |
|CPR         |
|BRO         |
|PLE         |
|BLU         |
|MAR         |
|BGE         |
|PNK         |
+------------+
only showing top 20 rows



In [74]:
df_units.select('veh_make_id').distinct().show(truncate=False)

+-----------------------------+
|veh_make_id                  |
+-----------------------------+
|UTILIMASTER                  |
|AMERICAN IRON HORSE          |
|ACURA                        |
|WHITE                        |
|MOTOR COACH MND INC          |
|PIERCE                       |
|PORSCHE                      |
|WHITEGMC                     |
|FREIGHTLINER                 |
|BUELL                        |
|STERLING                     |
|HYUNDAI                      |
|INTERNATIONAL                |
|PETER PIRSCH & SONS          |
|FIAT                         |
|GILLIG                       |
|NA                           |
|MIDLAND MANUFACTURING LIMITED|
|DATSUN                       |
|CAN-AM                       |
+-----------------------------+
only showing top 20 rows



In [76]:
df_units.select('veh_mod_id').distinct().show(truncate=False)

+-----------------------+
|veh_mod_id             |
+-----------------------+
|MDX                    |
|ASTRO                  |
|F SERIES               |
|ENTOURAGE              |
|MAZDA3I                |
|D-350                  |
|6 SERIES               |
|CAMARO                 |
|DYNASTY                |
|SHORT BED              |
|944                    |
|LIBERTY                |
|ESCALADE               |
|MONTE CARLO            |
|GRAND VOYAGER SPORT VAN|
|ENDEAVOR               |
|SL65AMG                |
|VOYAGER                |
|S60                    |
|124                    |
+-----------------------+
only showing top 20 rows



In [77]:
df_units.select('veh_body_styl_id').distinct().show(truncate=False)

+---------------------------------+
|veh_body_styl_id                 |
+---------------------------------+
|BUS                              |
|NA                               |
|VAN                              |
|PICKUP                           |
|SPORT UTILITY VEHICLE            |
|PASSENGER CAR, 4-DOOR            |
|FIRE TRUCK                       |
|TRUCK                            |
|UNKNOWN                          |
|AMBULANCE                        |
|POLICE CAR/TRUCK                 |
|MOTORCYCLE                       |
|YELLOW SCHOOL BUS                |
|POLICE MOTORCYCLE                |
|PASSENGER CAR, 2-DOOR            |
|TRUCK TRACTOR                    |
|FARM EQUIPMENT                   |
|NEV-NEIGHBORHOOD ELECTRIC VEHICLE|
|OTHER  (EXPLAIN IN NARRATIVE)    |
|NOT REPORTED                     |
+---------------------------------+



In [78]:
df_units.dropDuplicates(['veh_lic_state_id','vin','veh_mod_year','veh_make_id']).count()

159855

In [79]:
df_units.dropDuplicates(['veh_lic_state_id','vin','veh_mod_year','veh_make_id']).show(truncate=False)

+--------+--------+-------------+-------------+----------+----------------+-----------------+------------+------------+--------------------+--------------------+--------------------+---------------+----------+-----------------+--------------------+--------------------+-----------------+--------------+--------------------+-----------------+--------------+------------------+--------------------+--------------------+--------------------+--------------------+--------------------+---------------+---------------------+---------------+------------------+--------------+-------------+--------------+-------------+---------+
|CRASH_ID|UNIT_NBR| UNIT_DESC_ID|VEH_PARKED_FL|VEH_HNR_FL|VEH_LIC_STATE_ID|              VIN|VEH_MOD_YEAR|VEH_COLOR_ID|         VEH_MAKE_ID|          VEH_MOD_ID|    VEH_BODY_STYL_ID|EMER_RESPNDR_FL|  OWNR_ZIP|FIN_RESP_PROOF_ID|    FIN_RESP_TYPE_ID|  VEH_DMAG_AREA_1_ID|VEH_DMAG_SCL_1_ID|FORCE_DIR_1_ID|  VEH_DMAG_AREA_2_ID|VEH_DMAG_SCL_2_ID|FORCE_DIR_2_ID|VEH_INVENTORIED_FL|   

In [81]:
df_units.select('fin_resp_type_id').distinct().show(truncate=False)

+----------------------------------------+
|fin_resp_type_id                        |
+----------------------------------------+
|INSURANCE BINDER                        |
|LIABILITY INSURANCE POLICY              |
|NA                                      |
|CERTIFICATE OF SELF-INSURANCE           |
|CERTIFICATE OF DEPOSIT WITH COUNTY JUDGE|
|CERTIFICATE OF DEPOSIT WITH COMPTROLLER |
|SURETY BOND                             |
|PROOF OF LIABILITY INSURANCE            |
+----------------------------------------+



In [82]:
df_units.select('veh_dmag_scl_1_id').distinct().show(truncate=False)

+-----------------+
|veh_dmag_scl_1_id|
+-----------------+
|DAMAGED 4        |
|INVALID VALUE    |
|NA               |
|DAMAGED 5        |
|DAMAGED 1 MINIMUM|
|DAMAGED 3        |
|NO DAMAGE        |
|DAMAGED 7 HIGHEST|
|DAMAGED 2        |
|DAMAGED 6        |
+-----------------+



In [83]:
df_units.select('veh_dmag_scl_2_id').distinct().show(truncate=False)

+-----------------+
|veh_dmag_scl_2_id|
+-----------------+
|DAMAGED 4        |
|INVALID VALUE    |
|NA               |
|DAMAGED 5        |
|DAMAGED 1 MINIMUM|
|DAMAGED 3        |
|NO DAMAGE        |
|DAMAGED 7 HIGHEST|
|DAMAGED 2        |
|DAMAGED 6        |
+-----------------+



In [84]:
df_units.select('contrib_factr_1_id').distinct().show(truncate=False)

+----------------------------------------+
|contrib_factr_1_id                      |
+----------------------------------------+
|FAILED TO STOP FOR TRAIN                |
|OVERTAKE AND PASS INSUFFICIENT CLEARANCE|
|UNDER INFLUENCE - ALCOHOL               |
|WRONG SIDE - NOT PASSING                |
|TAKING MEDICATION (EXPLAIN IN NARRATIVE)|
|IMPROPER START FROM PARKED POSITION     |
|DISTRACTION IN VEHICLE                  |
|UNDER INFLUENCE - DRUG                  |
|FAILED TO YIELD ROW - TURN ON RED       |
|NA                                      |
|PARKED IN TRAFFIC LANE                  |
|ILL (EXPLAIN IN NARRATIVE)              |
|FAILED TO YIELD ROW - STOP SIGN         |
|UNSAFE SPEED                            |
|FAILED TO STOP AT PROPER PLACE          |
|DISREGARD STOP SIGN OR LIGHT            |
|FAILED TO YIELD ROW - PRIVATE DRIVE     |
|PASSED ON RIGHT SHOULDER                |
|DISREGARD TURN MARKS AT INTERSECTION    |
|FAILED TO PASS TO LEFT SAFELY           |
+----------

In [85]:
df_units.select('contrib_factr_2_id').distinct().show(truncate=False)

+----------------------------------------+
|contrib_factr_2_id                      |
+----------------------------------------+
|FAILED TO STOP FOR TRAIN                |
|OVERTAKE AND PASS INSUFFICIENT CLEARANCE|
|UNDER INFLUENCE - ALCOHOL               |
|WRONG SIDE - NOT PASSING                |
|TAKING MEDICATION (EXPLAIN IN NARRATIVE)|
|IMPROPER START FROM PARKED POSITION     |
|DISTRACTION IN VEHICLE                  |
|UNDER INFLUENCE - DRUG                  |
|FAILED TO YIELD ROW - TURN ON RED       |
|NA                                      |
|PARKED IN TRAFFIC LANE                  |
|ILL (EXPLAIN IN NARRATIVE)              |
|FAILED TO YIELD ROW - STOP SIGN         |
|UNSAFE SPEED                            |
|FAILED TO STOP AT PROPER PLACE          |
|FAILED TO YIELD ROW - PRIVATE DRIVE     |
|DISREGARD STOP SIGN OR LIGHT            |
|PASSED ON RIGHT SHOULDER                |
|DISREGARD TURN MARKS AT INTERSECTION    |
|FAILED TO PASS TO LEFT SAFELY           |
+----------

In [86]:
df_units.select('contrib_factr_p1_id').distinct().show(truncate=False)

+----------------------------------------+
|contrib_factr_p1_id                     |
+----------------------------------------+
|OVERTAKE AND PASS INSUFFICIENT CLEARANCE|
|UNDER INFLUENCE - ALCOHOL               |
|WRONG SIDE - NOT PASSING                |
|TAKING MEDICATION (EXPLAIN IN NARRATIVE)|
|IMPROPER START FROM PARKED POSITION     |
|DISTRACTION IN VEHICLE                  |
|UNDER INFLUENCE - DRUG                  |
|FAILED TO YIELD ROW - TURN ON RED       |
|NA                                      |
|PARKED IN TRAFFIC LANE                  |
|ILL (EXPLAIN IN NARRATIVE)              |
|FAILED TO YIELD ROW - STOP SIGN         |
|UNSAFE SPEED                            |
|FAILED TO STOP AT PROPER PLACE          |
|FAILED TO YIELD ROW - PRIVATE DRIVE     |
|DISREGARD STOP SIGN OR LIGHT            |
|PASSED ON RIGHT SHOULDER                |
|DISREGARD TURN MARKS AT INTERSECTION    |
|FAILED TO PASS TO LEFT SAFELY           |
|LOAD NOT SECURED                        |
+----------

In [87]:
df_units.select('tot_injry_cnt').distinct().show(truncate=False)

+-------------+
|tot_injry_cnt|
+-------------+
|1            |
|6            |
|3            |
|5            |
|15           |
|9            |
|4            |
|7            |
|10           |
|11           |
|2            |
|0            |
|31           |
|8            |
+-------------+



In [88]:
df_units.select('death_cnt').distinct().show(truncate=False)

+---------+
|death_cnt|
+---------+
|1        |
|3        |
|4        |
|2        |
|0        |
+---------+

